In [2]:
import shapely
import geopandas as gpd
import pandas as pd
import numpy as np


## Input
This is the only place where you have to change code if you want to change the dates or concentrations that you are filtering out, or if the column names of your files are different.

In [3]:
data_file_name = '../GAMA_allyears_restrictedcolumns.csv'

year_range = [90,10] # Only put the last two numbers of the year (e.g. 1990 > 90) Date format is "YYYY/MM/DD"
date_column_name = 'gm_samp_collection_date'

concentration_treshold = 10
concentration_column_name = 'gm_result'

lat_column_name = 'Latitude'
lon_column_name = 'Longitude'

# whenever want to add extra columns
keep_columns=[date_column_name, concentration_column_name]

---

### Load all zipcode polygons:

In [4]:
zip_gdf = gpd.read_file("zipcode_polygon.shp")[['ZCTA5CE10','geometry']]
zip_gdf.head()

,ZCTA5CE10,geometry
0,93201,"POLYGON ((-119.53768 35.92032, -119.53768 35.9..."
1,93202,"POLYGON ((-119.70291 36.31686, -119.70295 36.3..."
2,93203,"MULTIPOLYGON (((-118.77086 35.32751, -118.7708..."
3,93204,"MULTIPOLYGON (((-120.10873 36.05803, -120.1085..."
4,93206,"POLYGON ((-119.41234 35.48591, -119.41228 35.4..."


### Load and filter main data:
Here we are filtering by date, and by concentration

In [5]:
# Load full file:
df = pd.read_csv(data_file_name)

# Delete columns we don't need: 
df = df[[lat_column_name, lon_column_name]+ keep_columns]

# Filter rows by date:
dates_bool = pd.Series([(int(df[date_column_name][i][2:4]) >= year_range[0]) |
                        (int(df[date_column_name][i][2:4]) <= year_range[1])
                        for i in range(len(df))])
df = df[dates_bool]
df = df.reset_index().drop('index', axis=1)

# Filter rows by concentration:
conc_bool = pd.Series([df[concentration_column_name][i] >= concentration_treshold
                        for i in range(len(df))])
df = df[conc_bool]
df = df.reset_index().drop('index', axis=1)

df.head()

,Latitude,Longitude,gm_samp_collection_date,gm_result
0,37.683331,-121.76268,2010-06-29,11.10
1,37.683331,-121.76268,2010-07-19,10.30
2,37.683331,-121.76268,2010-07-26,10.30
3,37.683331,-121.76268,2010-08-23,10.99
4,37.683331,-121.76268,2010-09-07,10.77


### Convert the pd to gpd:
We need to convert the data from a pandas dataframe, to a geopandas dataframe. To do this, we have to convert the points (lat,lon) to "shapely" objects.

In [6]:
# Convert points to shapely objects, and add that as 'geometry' to the dataframe:
df['geometry'] = gpd.points_from_xy(df[lon_column_name], df[lat_column_name])

# Now just convert that Pandas dataframe to a GeoPandas dataframe:
data_gdf = gpd.GeoDataFrame(df, geometry='geometry').set_crs('EPSG:4326').to_crs(zip_gdf.crs)


In [7]:
data_gdf.head()

,Latitude,Longitude,gm_samp_collection_date,gm_result,geometry
0,37.683331,-121.76268,2010-06-29,11.10,POINT (-121.76268 37.68333)
1,37.683331,-121.76268,2010-07-19,10.30,POINT (-121.76268 37.68333)
2,37.683331,-121.76268,2010-07-26,10.30,POINT (-121.76268 37.68333)
3,37.683331,-121.76268,2010-08-23,10.99,POINT (-121.76268 37.68333)
4,37.683331,-121.76268,2010-09-07,10.77,POINT (-121.76268 37.68333)


### Now combine the zipcode data with the main data:
This will simply add a column with zip codes to the array. Any points that don't belong to any of the zipcodes will be removed from the dataframe 


In [8]:
filtered_gdf = data_gdf.sjoin(zip_gdf, how='inner').rename(columns={'ZCTA5CE10':'Zipcode'})

# Keep only the columns we care about:
filtered_gdf = filtered_gdf[['Zipcode']+ keep_columns]

filtered_gdf.head()

,Zipcode,gm_samp_collection_date,gm_result
14,93725,2010-05-21,11.0
17,93725,2010-01-25,11.0
18,93725,2010-07-26,11.0
1501,93725,2010-10-20,13.0
4628,93725,2007-10-01,11.0


In [9]:
# Save file:
filtered_gdf.to_csv(f'{data_file_name[:-4]}_zipcoded.csv', index=False)

In [10]:
print(f'{data_file_name[:-4]}_zipcoded.csv')

../GAMA_allyears_restrictedcolumns_zipcoded.csv
